In [198]:
# Usuals imports 
import numpy as np
import pandas as pd 
import requests
from time import sleep
import re
import urllib.parse

## Import hostels list

In [199]:
hostels_list = pd.read_json('src/hostels_list.json', encoding='utf-8')

In [200]:
def html_unicorn(sentence):
    foo = sentence.replace("\\/", "/").encode().decode('utf-8')
    foo = foo.replace(f'\n', '')
    return foo

In [201]:
for index in range(len(hostels_list)):
    if hostels_list['hotel_name'][index] == None:
        hostel_url = hostels_list['booking_page'][index]
        h_name = re.search(r'fr/(.*?).fr', hostel_url).group(1)
        h_name = h_name.replace('-', ' ')
        
        hostels_list['hotel_name'][index] = h_name
    

In [202]:
hostels_list = hostels_list.dropna().reset_index().drop(columns=['index'])
hostels_list = hostels_list.applymap(html_unicorn)

## Create lat / long columns

In [203]:
hostels_list['latitude'] = np.nan
hostels_list['longitude'] = np.nan

## Clear adress 

In [204]:
def clear_adress(sentence):
    adress = sentence
    try :
        todrop = re.findall(r', (?:...) arr.', adress)[0]
        adress = adress.replace(todrop, '')
    except:
        pass

    try :
        commas_index = re.search(r'[0-9]+(.*?) ', adress).end()-2
        adress = adress[:commas_index] + adress[commas_index+1:]
    except:
        pass

    return adress
    

In [205]:
#pandas shup up
pd.options.mode.chained_assignment = None 

for item in range(len(hostels_list)):
    address = clear_adress(hostels_list['adress'][item])

    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

    response = requests.get(url).json()
    try:
        hostels_list['latitude'][item] = response[0]["lat"]
        hostels_list['longitude'][item] = response[0]["lon"]
    except:
        pass

    print(f'{item}/{len(hostels_list)}')

0/1102
1/1102
2/1102
3/1102
4/1102
5/1102
6/1102
7/1102
8/1102
9/1102
10/1102
11/1102
12/1102
13/1102
14/1102
15/1102
16/1102
17/1102
18/1102
19/1102
20/1102
21/1102
22/1102
23/1102
24/1102
25/1102
26/1102
27/1102
28/1102
29/1102
30/1102
31/1102
32/1102
33/1102
34/1102
35/1102
36/1102
37/1102
38/1102
39/1102
40/1102
41/1102
42/1102
43/1102
44/1102
45/1102
46/1102
47/1102
48/1102
49/1102
50/1102
51/1102
52/1102
53/1102
54/1102
55/1102
56/1102
57/1102
58/1102
59/1102
60/1102
61/1102
62/1102
63/1102
64/1102
65/1102
66/1102
67/1102
68/1102
69/1102
70/1102
71/1102
72/1102
73/1102
74/1102
75/1102
76/1102
77/1102
78/1102
79/1102
80/1102
81/1102
82/1102
83/1102
84/1102
85/1102
86/1102
87/1102
88/1102
89/1102
90/1102
91/1102
92/1102
93/1102
94/1102
95/1102
96/1102
97/1102
98/1102
99/1102
100/1102
101/1102
102/1102
103/1102
104/1102
105/1102
106/1102
107/1102
108/1102
109/1102
110/1102
111/1102
112/1102
113/1102
114/1102
115/1102
116/1102
117/1102
118/1102
119/1102
120/1102
121/1102
122/1102
123

In [210]:
hostels_list.head()
hostels_list = hostels_list.dropna(subset=['latitude'])

hostels_list.to_csv(r'src/hostels_list_located.csv', index=False)